In [ ]:
import requests
import json

In [ ]:
gen9ou_data_url = "https://www.smogon.com/stats/2025-10/chaos/gen9vgc2025regh-1760.json"

response = requests.get(gen9ou_data_url)

if response.status_code == 200:
  print("Success")
  data = response.json()
  print(type(data))
  print(data.keys())
  # save to json file in drive
  with open("/content/drive/MyDrive/dataset/gen9vgc.json", "w") as f:
    json.dump(data, f)
else:
  print("Error")

Success
<class 'dict'>
dict_keys(['info', 'data'])


In [ ]:
top_pokemons = list(data['data'].keys())

print(len(top_pokemons))

267


In [ ]:
data['data']['Gholdengo'].keys()

dict_keys(['Raw count', 'Viability Ceiling', 'Abilities', 'Items', 'Spreads', 'Moves', 'Tera Types', 'Happiness', 'Teammates', 'Checks and Counters', 'usage'])

In [ ]:
gholdengo_items_data = data['data']['Gholdengo']['Items']
print(gholdengo_items_data)

{'lifeorb': 861.7188423146038, 'choicespecs': 609.4919105967721, 'choicescarf': 199.42559126199387, 'ironplate': 16.635621167753328, 'metalcoat': 196.09123501006968, 'airballoon': 1.4139308956223062, 'leftovers': 41.31455123711629, 'grassyseed': 75.65555847007197, 'sitrusberry': 18.19118979288491, 'focussash': 117.50882994095177, 'safetygoggles': 0.06836102261174942, 'nothing': 0.08806799344528621, 'assaultvest': 0.18583367901444892, 'mistyseed': 0.0, 'whiteherb': 0.939931924119684, 'spelltag': 0.03904007504574236, 'clearamulet': 0.18153462543967297, 'covertcloak': 6.654868763281733, 'ejectbutton': 0.0, 'ejectpack': 0.4283801100487843, 'weaknesspolicy': 1.983123513065799, 'colburberry': 0.026656716224256805, 'psychicseed': 0.2111738167904193, 'aguavberry': 1.1013412404281553e-12, 'rockyhelmet': 0.014898169296227004, 'mirrorherb': 0.05828363715713786, 'ironball': 0.18340471741431985, 'roomservice': 0.05375283629050709, 'shellbell': 0.001744346959279719, 'metalalloy': 1.0218912818899877,

In [ ]:
xd = sum(gholdengo_items_data[item] for item in gholdengo_items_data)
print(xd)

2150.0674341872827


In [ ]:
gholdengo_items_data = {item: value/xd for item, value in gholdengo_items_data.items() if value/xd > 0.2}

gholdengo_items_data

{'lifeorb': 0.40078689096573855, 'choicespecs': 0.283475718438179}

In [ ]:
def filter_by_usage(pokemon, field, filter=0.15):
  items_data = data['data'][pokemon][field]
  items_sum = sum(items_data[item] for item in items_data)
  return {key: value/items_sum for key, value in items_data.items() if value/items_sum > filter}


def filter_by_top_usage(pokemon, field, top_k=4):
  items_data = data['data'][pokemon][field]
  items_sum = sum(items_data[item] for item in items_data)
  items_data = {key: value/items_sum for key, value in items_data.items()}
  items_data_usage = sorted(items_data.items(), key=lambda x: x[1], reverse=True)
  items_data_usage = items_data_usage[:top_k]
  items_data = [key for key, value in items_data.items() if (key, value) in items_data_usage]
  return items_data


for pokemon in data['data']:
  pokemon_data = data['data'][pokemon]
  if 'Items' in pokemon_data:
    items_data = filter_by_usage(pokemon, 'Items')
    data['data'][pokemon]['Items'] = items_data

  if 'Spreads' in pokemon_data:
    spreads_data = filter_by_usage(pokemon, 'Spreads')
    data['data'][pokemon]['Spreads'] = spreads_data

  if 'Moves' in pokemon_data:
    moves_data = filter_by_top_usage(pokemon, 'Moves')
    data['data'][pokemon]['Moves'] = moves_data

  if 'Teammates' in pokemon_data:
    teammates_data = filter_by_top_usage(pokemon, 'Teammates', 6)
    data['data'][pokemon]['Teammates'] = teammates_data

  if 'Tera Types' in pokemon_data:
    terra_types_data = filter_by_usage(pokemon, 'Tera Types')
    data['data'][pokemon]['Tera Types'] = terra_types_data

  data['data'][pokemon].pop('Checks and Counters', None)
  data['data'][pokemon].pop('usage', None)
  data['data'][pokemon].pop('Happiness', None)
  data['data'][pokemon].pop('Viability Ceiling', None)
  data['data'][pokemon].pop('Raw count', None)


In [ ]:
data['data']['Incineroar']

{'Abilities': {'intimidate': 1849.89491476173, 'blaze': 1.8913728885438532},
 'Items': {'safetygoggles': 0.44532400149758045,
  'assaultvest': 0.21163262743097871},
 'Spreads': {},
 'Moves': ['partingshot', 'fakeout', 'flareblitz', 'knockoff'],
 'Tera Types': {'ghost': 0.5878268087727537, 'grass': 0.24986993222417095},
 'Teammates': ['Gholdengo',
  'Amoonguss',
  'Rillaboom',
  'Ursaluna-Bloodmoon',
  'Sneasler',
  'Dragonite']}

In [ ]:
with open("/content/drive/MyDrive/dataset/gen9vgc.json", "w") as f:
    json.dump(data, f)

In [ ]:
with open("/content/drive/MyDrive/dataset/gen9vgc.json", "r") as f:
    checkdata = json.load(f)

checkdata

{'info': {'metagame': 'gen9vgc2025regh',
  'cutoff': 1760.0,
  'cutoff deviation': 0,
  'team type': None,
  'number of battles': 650994},
 'data': {'Gholdengo': {'Abilities': {'goodasgold': 2150.0674341869494},
   'Items': {'lifeorb': 0.40078689096573855, 'choicespecs': 0.283475718438179},
   'Spreads': {'Modest:4/0/0/252/0/252': 0.1762777004844856},
   'Moves': ['makeitrain', 'shadowball', 'nastyplot', 'protect'],
   'Tera Types': {'water': 0.37287903306152614, 'steel': 0.45402448761654873},
   'Teammates': ['Incineroar',
    'Archaludon',
    'Sneasler',
    'Rillaboom',
    'Dragonite',
    'Ursaluna-Bloodmoon']},
  'Incineroar': {'Abilities': {'intimidate': 1849.89491476173,
    'blaze': 1.8913728885438532},
   'Items': {'safetygoggles': 0.44532400149758045,
    'assaultvest': 0.21163262743097871},
   'Spreads': {},
   'Moves': ['partingshot', 'fakeout', 'flareblitz', 'knockoff'],
   'Tera Types': {'ghost': 0.5878268087727537, 'grass': 0.24986993222417095},
   'Teammates': ['Ghold

In [ ]:
len(checkdata['data'])

267

In [ ]:
split = 267//3

print(split)

89


In [ ]:
jona_dataset = []
stan_dataset = []
ian_dataset = []

counter = 0
for pokemon in checkdata['data']:
  if counter < 89:
    jona_dataset.append({pokemon: checkdata['data'][pokemon]})
  elif counter < 178:
    stan_dataset.append({pokemon: checkdata['data'][pokemon]})
  else:
    ian_dataset.append({pokemon: checkdata['data'][pokemon]})

  counter += 1

print(len(jona_dataset), len(stan_dataset), len(ian_dataset))


89 89 89


In [ ]:
jona_dataset

[{'Gholdengo': {'Abilities': {'goodasgold': 2150.0674341869494},
   'Items': {'lifeorb': 0.40078689096573855, 'choicespecs': 0.283475718438179},
   'Spreads': {'Modest:4/0/0/252/0/252': 0.1762777004844856},
   'Moves': ['makeitrain', 'shadowball', 'nastyplot', 'protect'],
   'Tera Types': {'water': 0.37287903306152614, 'steel': 0.45402448761654873},
   'Teammates': ['Incineroar',
    'Archaludon',
    'Sneasler',
    'Rillaboom',
    'Dragonite',
    'Ursaluna-Bloodmoon']}},
 {'Incineroar': {'Abilities': {'intimidate': 1849.89491476173,
    'blaze': 1.8913728885438532},
   'Items': {'safetygoggles': 0.44532400149758045,
    'assaultvest': 0.21163262743097871},
   'Spreads': {},
   'Moves': ['partingshot', 'fakeout', 'flareblitz', 'knockoff'],
   'Tera Types': {'ghost': 0.5878268087727537, 'grass': 0.24986993222417095},
   'Teammates': ['Gholdengo',
    'Amoonguss',
    'Rillaboom',
    'Ursaluna-Bloodmoon',
    'Sneasler',
    'Dragonite']}},
 {'Ursaluna-Bloodmoon': {'Abilities': {'min

In [ ]:
with open("/content/drive/MyDrive/dataset/jona_dataset.json", "w") as f:
  json.dump(jona_dataset, f)

with open("/content/drive/MyDrive/dataset/stan_dataset.json", "w") as f:
  json.dump(stan_dataset, f)

with open("/content/drive/MyDrive/dataset/ian_dataset.json", "w") as f:
  json.dump(ian_dataset, f)

In [ ]:
import json

with open("/content/drive/MyDrive/dataset/jona_dataset.json", "r") as f:
  pkmn_dataset = json.load(f)


In [ ]:
for pokemon in pkmn_dataset:
  print(pokemon)

{'Gholdengo': {'Abilities': {'goodasgold': 2150.0674341869494}, 'Items': {'lifeorb': 0.40078689096573855, 'choicespecs': 0.283475718438179}, 'Spreads': {'Modest:4/0/0/252/0/252': 0.1762777004844856}, 'Moves': ['makeitrain', 'shadowball', 'nastyplot', 'protect'], 'Tera Types': {'water': 0.37287903306152614, 'steel': 0.45402448761654873}, 'Teammates': ['Incineroar', 'Archaludon', 'Sneasler', 'Rillaboom', 'Dragonite', 'Ursaluna-Bloodmoon']}}
{'Incineroar': {'Abilities': {'intimidate': 1849.89491476173, 'blaze': 1.8913728885438532}, 'Items': {'safetygoggles': 0.44532400149758045, 'assaultvest': 0.21163262743097871}, 'Spreads': {}, 'Moves': ['partingshot', 'fakeout', 'flareblitz', 'knockoff'], 'Tera Types': {'ghost': 0.5878268087727537, 'grass': 0.24986993222417095}, 'Teammates': ['Gholdengo', 'Amoonguss', 'Rillaboom', 'Ursaluna-Bloodmoon', 'Sneasler', 'Dragonite']}}
{'Ursaluna-Bloodmoon': {'Abilities': {'mindseye': 1601.977697552006}, 'Items': {'lifeorb': 0.2788775780342926, 'assaultvest':

### Only on google colab

In [ ]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY_1")

# MAKING DATASET

In [ ]:
%pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 4.3 MB/s eta 0:00:00


### Contoh pembuatan satu bagian dataset -> menggunakan jona_dataet.json

In [ ]:
import time
import random
import json
from groq import Groq
from pydantic import BaseModel, Field
from typing import Literal
from google.colab import userdata

# load dataset
with open("/content/drive/MyDrive/dataset/jona_dataset.json", "r") as f:
  pkmn_dataset = json.load(f)


# mendefinisikan data schema untuk enable structured output

class AnalysisOutput(BaseModel):
    short_summary: str = Field(..., description="Section 1: Short Summary.")
    full_analysis: str = Field(..., description="Section 2: Full Competitive Analysis.")
    matchup_overview: str = Field(..., description="Section 3: Matchup Overview.")
    role_description: str = Field(..., description="Section 4: Role Description.")
    synergy_breakdown: str = Field(..., description="Section 5: Synergy Breakdown.")
    recommended_sets: str = Field(..., description="Section 6: Recommended Sets.")
    weaknesses_and_counters: str = Field(..., description="Section 7: Weaknesses & Counters.")
    sample_team_concept: str = Field(..., description="Section 8: Sample Team Concept.")


class QuestionAnswer(BaseModel):
    input: str = Field(..., description="The user prompt to execute by the model + Pokémon JSON data.")
    output: AnalysisOutput = Field(..., description="The expected structured 8-section output.")


GROQ_API_KEY = userdata.get("GROQ_APIKEY")
print(GROQ_API_KEY)
client = Groq(api_key=GROQ_API_KEY)

random.seed(42)

checkpoint = 10
all_qa = []

instruction_random = [
    "Write the ",
    "Generate the ",
    "Create the ",
    "Produce the ",
    "Draft the ",
    "Write a detailed ",
    "Provide the ",
]

def safe_extract_text(response):
    """Extracts the text from Groq response safely."""
    try:
        return response.choices[0].message.content
    except:
        return response.choices[0].message.content


def generate_with_retry(prompt, analysis_output_schema, max_attempts=10):
    """Retry Groq call with exponential backoff."""
    wait = 30

    for attempt in range(max_attempts):
        try:
            return client.chat.completions.create(
              model="moonshotai/kimi-k2-instruct-0905",
              messages=prompt,
              response_format={
                  "type": "json_schema",
                  "json_schema": {
                      "name": "competitive_pokemon_analysis",
                      "schema": analysis_output_schema
                  }
              }
            )
        except Exception as e:
            print(f"[ERROR] Attempt {attempt+1}/{max_attempts}: {e}")
            print(f"Waiting {wait} seconds...\n")
            time.sleep(wait)
            wait = min(wait * 1.1, 180)  # cap at 3 minutes

    raise RuntimeError("Max retry attempts exceeded.")


for i, pokemon in enumerate(pkmn_dataset, 1):
    prompt = f"""
    You are a competitive Pokémon analyst specializing in VGC.

    Using ONLY the information in the JSON, produce an output that matches EXACTLY this JSON structure:

    {{
      "short_summary": "",
      "full_analysis": "",
      "matchup_overview": "",
      "role_description": "",
      "synergy_breakdown": "",
      "recommended_sets": "",
      "weaknesses_and_counters": "",
      "sample_team_concept": ""
    }}

    Fill each field with a full detailed paragraph.

    JSON DATA:
    <POKEMON_JSON>{json.dumps(pokemon, ensure_ascii=False)}</POKEMON_JSON>
    """
    message = [
        {
          "role": "system",
          "content": (
              "You are a competitive Pokémon VGC analysis model. "
              "Your job is to read structured JSON data about a Pokémon and generate a detailed competitive analysis. "
              "Follow the user's requested format exactly, use only the provided JSON, and do not hallucinate."
          ),
        },
        {
            "role": "user",
            "content": prompt,
        }
    ]
    response = generate_with_retry(message, analysis_output_schema=AnalysisOutput.model_json_schema())
    raw_text = safe_extract_text(response)

    analysis_output_instance = AnalysisOutput.model_validate_json(raw_text)
    master_dict = analysis_output_instance.model_dump()

    pokemon_json = json.dumps(pokemon)

    for section_name, section_text in master_dict.items():
        section_prompt = (
            random.choice(instruction_random)
            + f"'{section_name}' section using only the data given.\n\n"
            + f"<DATA>{pokemon_json}</DATA>"
        )

        current_answer = {
            "pokemon": list(pokemon.keys())[0],
            "input": section_prompt,
            "output": f"<OUTPUT>{section_text}</OUTPUT>",
        }
        all_qa.append(current_answer)

        # Save checkpoints
        if len(all_qa) >= checkpoint:
            checkpoint += 10
            with open("stan_dataset_cleaned.json", "w", encoding="utf-8") as f:
                json.dump(all_qa, f, ensure_ascii=False, indent=4)

        print(f"Processing done: {i} / {len(pkmn_dataset)}")
        print(current_answer)
        print()

    time.sleep(1)  # only ONE sleep per Pokémon


with open("/content/drive/MyDrive/dl_dataset/jona_dataset_cleaned.json", "w") as f:
  json.dump(all_qa, f, ensure_ascii=False, indent=4)

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
Processing done: 1 / 89
{'pokemon': 'Gholdengo', 'input': 'Write a detailed \'short_summary\' section using only the data given.\n\n<DATA>{"Gholdengo": {"Abilities": {"goodasgold": 2150.0674341869494}, "Items": {"lifeorb": 0.40078689096573855, "choicespecs": 0.283475718438179}, "Spreads": {"Modest:4/0/0/252/0/252": 0.1762777004844856}, "Moves": ["makeitrain", "shadowball", "nastyplot", "protect"], "Tera Types": {"water": 0.37287903306152614, "steel": 0.45402448761654873}, "Teammates": ["Incineroar", "Archaludon", "Sneasler", "Rillaboom", "Dragonite", "Ursaluna-Bloodmoon"]}}</DATA>', 'output': '<OUTPUT>Gholdengo is a premier special attacker in VGC, leveraging its signature ability Good as Gold—an immunity to status moves—to serve as an unflinching offensive pivot. With a Life Orb or Choice Specs set, it fires off high-powered STAB Make It Rain and Shadow Ball while setting up Nasty Plot to break through bulky cores. Its flexible 

In [ ]:
with open("/content/drive/MyDrive/dl_dataset/stan_dataset_cleaned.json", "w") as f:
  json.dump(all_qa, f, ensure_ascii=False, indent=4)